<a href="https://www.spe.org/events/en/2022/conference/22apog/asia-pacific-oil-and-gas-conference-and-exhibition.html"><img src = "https://www.spe.org/binaries/content/gallery/specms/speevents/organization-logos/spe-logo-2020.png" width = 200> 

<h1 align=center><font size = 5>Prediction of Recovery Factor using Machine Learning Methods</font></h1>

<h1 align=center><font size = 4> Munish Kumar, Kannapan Swaminathan</font></h1>
<h1 align=center><font size = 4> Part 5: Prediction of Recovery Factor</font></h1>
<h1 align=center><font size = 3> ERCE 2022 </font></h1>

#### Libraries

In [ ]:
# Only install the following libraries if you dont have it, otherwise leave it commented out

#!conda install -c anaconda natsort --yes
#!conda install -c anaconda xlrd --yes
#!pip install pycaret[full] --user
#!pip install mlflow --user
#!pip install tune-sklearn ray[tune] --user
#!pip install optuna -- user
#!pip install hyperopt --user

# General Libraries
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import NullFormatter
import time
import re
import requests
import pickle
import seaborn as sns
import os
import glob
import sys
from natsort import natsorted
sns.set()

import plotly.graph_objects as go
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

# Sklearn Liraries
from sklearn import preprocessing

import datetime
from datetime import timedelta, date 
start = time.time()
%matplotlib inline

import ray
from ray import tune

# Forces the print statement to show everything and not truncate
# np.set_printoptions(threshold=sys.maxsize) 
print('Libraries imported')

In [ ]:
# Function to read out file
def file_out(df, dir_name, base_filename, filename_suffix):
    try:
        csvs_sht = os.path.join(dir_name, base_filename + "." + filename_suffix)
        df.to_csv(csvs_sht, index = False, header=True)
        print ("Final File Extract Produced")
    except IOError:
        print("I/O error")
    return

###### Declare some global variables

In [ ]:
#Input Data
#dir_name = r'C:\Users\kswaminathan\OneDrive\01_KannaLibrary\15_Analogs'
dir_name = r'C:\Users\mkumar\Documents\GitHub\SPE_Paper\Final'
filename_suffix = 'csv'

##### Read in the data 

In [ ]:
skiprows = 0
dfblind = pd.read_excel("BlindTest_SSOIL.xlsx", sheet_name='Inputs', thousands=',', skiprows = skiprows)
#dfblind = dfblind.loc[:, ~df.columns.str.contains('^Unnamed')] 
#dfblind_ori = dfblind.copy()
dfblind.drop('307_Recovery factor (ultimate oil) (%)', axis=1, inplace=True)
dfblind.dropna(axis = 0, inplace=True)

dfblind.head(10)

## Load the Model

In [ ]:
from pycaret.regression import load_model
from pycaret.regression import predict_model

from functools import reduce

final_blend = load_model('Blended_model_21042022')
final_db = load_model('Boosted_CatBoost_21042022')
final_gbr = load_model('Boosted_GBR_21042022')
final_rf = load_model('Boosted_RF_21042022')

In [ ]:
print(final_blend)

In [ ]:
BlindPredict1 = predict_model(final_blend, data=dfblind, round=2)
BlindPredict2 = predict_model(final_db, data=dfblind, round=2)
BlindPredict3 = predict_model(final_gbr, data=dfblind, round=2)
BlindPredict4 = predict_model(final_rf, data=dfblind, round=2)

BlindPredict1 = BlindPredict1.rename(columns={'Label': 'Blended Predicted Recovery factor (ultimate oil) (%)'
                                 })
BlindPredict2 = BlindPredict2.rename(columns={'Label': 'CatBoost Predicted Recovery factor (ultimate oil) (%)'
                                 })
BlindPredict3 = BlindPredict3.rename(columns={'Label': 'Gradient Boost Predicted Recovery factor (ultimate oil) (%)'
                                 })
BlindPredict4 = BlindPredict4.rename(columns={'Label': 'Random Forest Predicted Recovery factor (ultimate oil) (%)'
                                 })

BlindPredict_df = [BlindPredict1, BlindPredict2, BlindPredict3, BlindPredict4]

BlindPredict = pd.concat(BlindPredict_df, axis = 0, join = 'outer').drop_duplicates()
BlindPredict = BlindPredict.reset_index(drop=True)

BlindPredict

In [ ]:
aggregation_functions = {
    'Blended Predicted Recovery factor (ultimate oil) (%)': 'sum', 
    'CatBoost Predicted Recovery factor (ultimate oil) (%)': 'sum',
    'Gradient Boost Predicted Recovery factor (ultimate oil) (%)': 'sum',
    'Random Forest Predicted Recovery factor (ultimate oil) (%)': 'sum',
}

labels = dfblind.columns.tolist()

BlindPredict_new = BlindPredict.groupby(labels).aggregate(aggregation_functions)

file_out(BlindPredict, dir_name, 'Blind_Test_Out' , filename_suffix)

BlindPredict_new